In [4]:
import json
from pyspark.mllib.feature import Word2Vec


In [5]:
with open('/home/han/.api_key/awsaccesskey.json') as f:
    key= json.load(f)
    
    access= key['access-key']
    secret = key['secret-access-key']

# test

In [6]:
data = sc.textFile('s3n://'+access+':'+secret +'@sparkdatasets/text8_lines')


In [23]:
data.take(5)

[u'anarchism originated as a term of abuse first used against early working class radicals including the',
 u'diggers of the english revolution and the sans culottes of the french revolution whilst the term',
 u'is still used in a pejorative way to describe any act that used violent means to',
 u'destroy the organization of society it has also been taken up as a positive label by',
 u'self defined anarchists the word anarchism is derived from the greek without archons ruler chief king']

In [7]:
word2vec = Word2Vec()


In [16]:
word_rdd = data.map(lambda x: x.split())

In [17]:
word2vec.fit(word_rdd)

In [10]:
data.getNumPartitions()

2

In [24]:
w2v_mod = _17

In [27]:
w2v_mod.findSynonyms('greek', 10)

[(u'mythology', 1.6330972853653365),
 (u'ancient', 1.6254558742548857),
 (u'latin', 1.5719785017791681),
 (u'koine', 1.5699189217969287),
 (u'phoenician', 1.5505149126688398),
 (u'sanskrit', 1.494935579959364),
 (u'akkadian', 1.4940735645984307),
 (u'constantinopolis', 1.4848666144920364),
 (u'persian', 1.4798156868382155),
 (u'macedonian', 1.4750030810843235)]

# W2V tweet

In [28]:
from pyspark import SparkContext
import json
import nltk
import re
from collections import Counter, defaultdict

In [29]:
REGEX = u"[\U00002712\U00002714\U00002716\U0000271d\U00002721\U00002728\U00002733\U00002734\U00002744\U00002747\U0000274c\U0000274e\U00002753-\U00002755\U00002757\U00002763\U00002764\U00002795-\U00002797\U000027a1\U000027b0\U000027bf\U00002934\U00002935\U00002b05-\U00002b07\U00002b1b\U00002b1c\U00002b50\U00002b55\U00003030\U0000303d\U0001f004\U0001f0cf\U0001f170\U0001f171\U0001f17e\U0001f17f\U0001f18e\U0001f191-\U0001f19a\U0001f201\U0001f202\U0001f21a\U0001f22f\U0001f232-\U0001f23a\U0001f250\U0001f251\U0001f300-\U0001f321\U0001f324-\U0001f393\U0001f396\U0001f397\U0001f399-\U0001f39b\U0001f39e-\U0001f3f0\U0001f3f3-\U0001f3f5\U0001f3f7-\U0001f4fd\U0001f4ff-\U0001f53d\U0001f549-\U0001f54e\U0001f550-\U0001f567\U0001f56f\U0001f570\U0001f573-\U0001f579\U0001f587\U0001f58a-\U0001f58d\U0001f590\U0001f595\U0001f596\U0001f5a5\U0001f5a8\U0001f5b1\U0001f5b2\U0001f5bc\U0001f5c2-\U0001f5c4\U0001f5d1-\U0001f5d3\U0001f5dc-\U0001f5de\U0001f5e1\U0001f5e3\U0001f5ef\U0001f5f3\U0001f5fa-\U0001f64f\U0001f680-\U0001f6c5\U0001f6cb-\U0001f6d0\U0001f6e0-\U0001f6e5\U0001f6e9\U0001f6eb\U0001f6ec\U0001f6f0\U0001f6f3\U0001f910-\U0001f918\U0001f980-\U0001f984\U0001f9c0\U00003297\U00003299\U000000a9\U000000ae\U0000203c\U00002049\U00002122\U00002139\U00002194-\U00002199\U000021a9\U000021aa\U0000231a\U0000231b\U00002328\U00002388\U000023cf\U000023e9-\U000023f3\U000023f8-\U000023fa\U000024c2\U000025aa\U000025ab\U000025b6\U000025c0\U000025fb-\U000025fe\U00002600-\U00002604\U0000260e\U00002611\U00002614\U00002615\U00002618\U0000261d\U00002620\U00002622\U00002623\U00002626\U0000262a\U0000262e\U0000262f\U00002638-\U0000263a\U00002648-\U00002653\U00002660\U00002663\U00002665\U00002666\U00002668\U0000267b\U0000267f\U00002692-\U00002694\U00002696\U00002697\U00002699\U0000269b\U0000269c\U000026a0\U000026a1\U000026aa\U000026ab\U000026b0\U000026b1\U000026bd\U000026be\U000026c4\U000026c5\U000026c8\U000026ce\U000026cf\U000026d1\U000026d3\U000026d4\U000026e9\U000026ea\U000026f0-\U000026f5\U000026f7-\U000026fa\U000026fd\U00002702\U00002705\U00002708-\U0000270d\U0000270f]|[#]\U000020e3|[*]\U000020e3|[0]\U000020e3|[1]\U000020e3|[2]\U000020e3|[3]\U000020e3|[4]\U000020e3|[5]\U000020e3|[6]\U000020e3|[7]\U000020e3|[8]\U000020e3|[9]\U000020e3|\U0001f1e6[\U0001f1e8-\U0001f1ec\U0001f1ee\U0001f1f1\U0001f1f2\U0001f1f4\U0001f1f6-\U0001f1fa\U0001f1fc\U0001f1fd\U0001f1ff]|\U0001f1e7[\U0001f1e6\U0001f1e7\U0001f1e9-\U0001f1ef\U0001f1f1-\U0001f1f4\U0001f1f6-\U0001f1f9\U0001f1fb\U0001f1fc\U0001f1fe\U0001f1ff]|\U0001f1e8[\U0001f1e6\U0001f1e8\U0001f1e9\U0001f1eb-\U0001f1ee\U0001f1f0-\U0001f1f5\U0001f1f7\U0001f1fa-\U0001f1ff]|\U0001f1e9[\U0001f1ea\U0001f1ec\U0001f1ef\U0001f1f0\U0001f1f2\U0001f1f4\U0001f1ff]|\U0001f1ea[\U0001f1e6\U0001f1e8\U0001f1ea\U0001f1ec\U0001f1ed\U0001f1f7-\U0001f1fa]|\U0001f1eb[\U0001f1ee-\U0001f1f0\U0001f1f2\U0001f1f4\U0001f1f7]|\U0001f1ec[\U0001f1e6\U0001f1e7\U0001f1e9-\U0001f1ee\U0001f1f1-\U0001f1f3\U0001f1f5-\U0001f1fa\U0001f1fc\U0001f1fe]|\U0001f1ed[\U0001f1f0\U0001f1f2\U0001f1f3\U0001f1f7\U0001f1f9\U0001f1fa]|\U0001f1ee[\U0001f1e8-\U0001f1ea\U0001f1f1-\U0001f1f4\U0001f1f6-\U0001f1f9]|\U0001f1ef[\U0001f1ea\U0001f1f2\U0001f1f4\U0001f1f5]|\U0001f1f0[\U0001f1ea\U0001f1ec-\U0001f1ee\U0001f1f2\U0001f1f3\U0001f1f5\U0001f1f7\U0001f1fc\U0001f1fe\U0001f1ff]|\U0001f1f1[\U0001f1e6-\U0001f1e8\U0001f1ee\U0001f1f0\U0001f1f7-\U0001f1fb\U0001f1fe]|\U0001f1f2[\U0001f1e6\U0001f1e8-\U0001f1ed\U0001f1f0-\U0001f1ff]|\U0001f1f3[\U0001f1e6\U0001f1e8\U0001f1ea-\U0001f1ec\U0001f1ee\U0001f1f1\U0001f1f4\U0001f1f5\U0001f1f7\U0001f1fa\U0001f1ff]|\U0001f1f4\U0001f1f2|\U0001f1f5[\U0001f1e6\U0001f1ea-\U0001f1ed\U0001f1f0-\U0001f1f3\U0001f1f7-\U0001f1f9\U0001f1fc\U0001f1fe]|\U0001f1f6\U0001f1e6|\U0001f1f7[\U0001f1ea\U0001f1f4\U0001f1f8\U0001f1fa\U0001f1fc]|\U0001f1f8[\U0001f1e6-\U0001f1ea\U0001f1ec-\U0001f1f4\U0001f1f7-\U0001f1f9\U0001f1fb\U0001f1fd-\U0001f1ff]|\U0001f1f9[\U0001f1e6\U0001f1e8\U0001f1e9\U0001f1eb-\U0001f1ed\U0001f1ef-\U0001f1f4\U0001f1f7\U0001f1f9\U0001f1fb\U0001f1fc\U0001f1ff]|\U0001f1fa[\U0001f1e6\U0001f1ec\U0001f1f2\U0001f1f8\U0001f1fe\U0001f1ff]|\U0001f1fb[\U0001f1e6\U0001f1e8\U0001f1ea\U0001f1ec\U0001f1ee\U0001f1f3\U0001f1fa]|\U0001f1fc[\U0001f1eb\U0001f1f8]|\U0001f1fd\U0001f1f0|\U0001f1fe[\U0001f1ea\U0001f1f9]|\U0001f1ff[\U0001f1e6\U0001f1f2\U0001f1fc]|[0-9*#]\ufe0f\u20e3"


In [31]:
def tweet_process(tweet):
    KEY = 'text'
    try:
        tw = json.loads(tweet.strip())
        if KEY not in tw or tw['lang']!= 'en':
            return None
        return tw

    except Exception as e:
        return None


In [32]:

def emoji_preprocess(tweet):
    # add space before and after space
    for emoji in re.findall(REGEX, tweet):
        tweet = tweet.replace(emoji, ' ' + emoji + ' ')

    # tokenize and remove rt and @ and https://

    tweet_token = ['<s>'] + [ wd for wd in tweet.strip('rt').split() if not wd.startswith('@') and not wd.startswith('http') and not wd.startswith('#') ] + ['</s>']

    return tweet_token


In [48]:
tweets =  sc.textFile('data/twitter_dump.txt')\
.filter(lambda tw: len(tw)>1)\
.filter(lambda tw: 'created_at' in tw)\
.map(tweet_process)\
.filter(lambda tw: tw != None)\
.map(lambda tw: tw['text'].lower() )\


In [49]:
tweets_tokens = tweets.map(emoji_preprocess)

In [51]:
tweet_w2v = word2vec.fit(tweets_tokens)

In [69]:
tweet_w2v.findSynonyms(u'trump', 20)


[(u'trump?', 1.0815301565615842),
 (u'trump,', 1.0735135042143815),
 (u'donald', 1.0317948790884357),
 (u"'hitler'", 1.0292268032071032),
 (u'straight-up', 1.0274710243143497),
 (u'trum', 0.99784914060017615),
 (u'trump.', 0.99622355356754166),
 (u"'hitler':", 0.99603074272405312),
 (u'tr', 0.99365224296006394),
 (u'trump".', 0.98662279287440735),
 (u'(r)', 0.98578550264575093),
 (u'trumps', 0.96878778985293468),
 (u"'ted", 0.96328434632703219),
 (u'snl', 0.96268034837961258),
 (u'anti-racism', 0.95703735025046066),
 (u'expands', 0.95609231328991073),
 (u'34%', 0.95405627436813578),
 (u'convention', 0.95176192456147757),
 (u'duck', 0.94694444899639196),
 (u'ump', 0.94651519183095567)]

In [50]:
tweets_tokens.count()

435305

In [66]:
tweet_w2v.getVectors()

KeyboardInterrupt: 